# Test Bed


In [11]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import numpy as np

from src.spectra.gen_spectra import gen_spectrum
from src.types.objects import Spectrum
from src.utils import insort_by_index, make_sparse_array
from src.scoring import scoring

In [2]:
DEAPNFEANTTIGRIRFH = Spectrum(spectrum=[70.06517028808594, 74.06005859375, 84.04444122314453, 87.05548095703125, 88.03839111328125, 102.05506896972656, 120.08072662353516, 201.12315368652344, 212.1022186279297, 217.08192443847656, 245.07611083984375, 316.1118469238281, 585.966796875, 591.6425170898438, 591.9791870117188, 592.3104248046875, 607.8389282226562, 615.3208618164062, 615.6561279296875, 643.3582763671875, 643.8550415039062, 707.87841796875, 708.3804931640625, 781.4142456054688, 838.4376220703125], abundance=[2547.683349609375, 730.1649169921875, 1565.4937744140625, 958.437744140625, 917.452392578125, 3846.709716796875, 2452.79736328125, 987.6354370117188, 1954.6824951171875, 3886.33349609375, 6083.39990234375, 712.1141357421875, 843.4388427734375, 3621.18359375, 3374.97021484375, 1484.1064453125, 1291.126953125, 968.104248046875, 1789.317138671875, 2742.34521484375, 1139.092041015625, 950.854736328125, 762.1254272460938, 748.9049072265625, 758.2952880859375], total_intensity=47116.769775390625, ms_level=2, scan_number=17, precursor_mass=1045.526854, file_name='/Users/zacharymcgrath/Desktop/nod2 data/single/singleRealSpectrum.mzml', id='NOD2_E3.9670.9702.4.pkl')
DEAPNFEANTTIGRIRFHs = 'DEAPNFEANTTIGRIRFH'

DLQTLEVE = Spectrum(spectrum=[72.08074951171875, 84.04391479492188, 86.0966796875, 101.07064056396484, 102.05500793457031, 130.04966735839844, 132.10191345214844, 148.06016540527344, 183.11273193359375, 197.12803649902344, 201.1233673095703, 212.10264587402344, 215.13906860351562, 229.11734008789062, 243.1343536376953, 247.12913513183594, 357.1775207519531, 358.16217041015625, 440.2137756347656, 458.22381591796875, 472.23712158203125, 553.2945556640625, 571.3069458007812, 682.3330688476562, 700.3469848632812], abundance=[11277.177734375, 2677.902099609375, 3753.010498046875, 1874.318359375, 3304.14306640625, 4072.84765625, 1058.6568603515625, 18233.23046875, 1099.9212646484375, 891.7163696289062, 4566.39990234375, 767.3123779296875, 1045.271728515625, 1623.881103515625, 856.7132568359375, 3777.787109375, 1373.5948486328125, 1266.3497314453125, 1810.636962890625, 2190.313232421875, 787.8353881835938, 2011.8814697265625, 1665.766357421875, 1007.9365234375, 2019.712646484375], total_intensity=75014.31701660156, ms_level=2, scan_number=10, precursor_mass=473.740315, file_name='/Users/zacharymcgrath/Desktop/nod2 data/single/singleRealSpectrum.mzml', id='NOD2_E3.11427.11516.2.pkl')
DLQTLEVEs = 'DLQTLEVE'

DLQTLALWSRM = Spectrum(spectrum=[84.04356384277344, 86.095703125, 101.07088470458984, 201.12228393554688, 202.1270751953125, 212.10423278808594, 229.1165008544922, 246.14280700683594, 306.16021728515625, 357.17578125, 393.1902770996094, 430.2205505371094, 440.2127685546875, 458.2236022949219, 553.2975463867188, 579.2705078125, 624.333251953125, 642.3447875976562, 658.3429565429688, 692.3502197265625, 763.3924560546875, 764.3958129882812, 876.477783203125, 977.5200805664062, 978.5184326171875], abundance=[907.1680908203125, 2089.55615234375, 1746.5712890625, 1797.59326171875, 517.5567626953125, 576.9080200195312, 1723.9498291015625, 829.689208984375, 973.6568603515625, 1102.828857421875, 1670.658203125, 730.5213623046875, 950.6202392578125, 1065.868896484375, 1333.5362548828125, 1928.1298828125, 525.9630126953125, 818.0933227539062, 606.9487915039062, 1494.930908203125, 3529.6337890625, 817.701171875, 911.8171997070312, 1712.318603515625, 685.1273193359375], total_intensity=31047.347290039062, ms_level=2, scan_number=10129, precursor_mass=667.350088, file_name='/Users/zacharymcgrath/Desktop/nod2 data/single/singleRealSpectrum.mzml', id='scan=10130')
DLQTLALWSRMs = 'DLQTLALWSRM'

DLTEYLSRFGEVV = Spectrum(spectrum=[70.06398010253906, 72.08049774169922, 86.09703063964844, 118.08600616455078, 136.07492065429688, 187.0723419189453, 201.12191772460938, 217.15528869628906, 229.1175994873047, 230.12203979492188, 312.15435791015625, 330.1663818359375, 334.13720703125, 441.199951171875, 459.2086486816406, 647.3054809570312, 647.8048095703125, 656.810791015625, 746.8765869140625, 793.4156494140625, 906.4981079101562, 978.4926147460938, 1069.56689453125, 1070.5753173828125, 1311.6202392578125], abundance=[855.636474609375, 3774.81884765625, 812.15185546875, 533.6064453125, 490.1760559082031, 414.03570556640625, 1025.067138671875, 792.000244140625, 1133.7769775390625, 346.464599609375, 612.9151000976562, 488.3008117675781, 484.1494140625, 353.00909423828125, 504.1302185058594, 382.40264892578125, 361.7584228515625, 359.9140319824219, 428.7555847167969, 540.8904418945312, 452.9610290527344, 643.1677856445312, 458.5113525390625, 399.8130187988281, 694.0618896484375], total_intensity=17342.475189208984, ms_level=2, scan_number=11409, precursor_mass=764.388677, file_name='/Users/zacharymcgrath/Desktop/nod2 data/single/singleRealSpectrum.mzml', id='scan=11410')
DLTEYLSRFGEVVs = 'DLTEYLSRFGEVV'

DLQTLEVE = Spectrum(spectrum=[72.08074951171875, 84.04391479492188, 86.0966796875, 101.07064056396484, 102.05500793457031, 130.04966735839844, 132.10191345214844, 148.06016540527344, 183.11273193359375, 197.12803649902344, 201.1233673095703, 212.10264587402344, 215.13906860351562, 229.11734008789062, 243.1343536376953, 247.12913513183594, 357.1775207519531, 358.16217041015625, 440.2137756347656, 458.22381591796875, 472.23712158203125, 553.2945556640625, 571.3069458007812, 682.3330688476562, 700.3469848632812], abundance=[11277.177734375, 2677.902099609375, 3753.010498046875, 1874.318359375, 3304.14306640625, 4072.84765625, 1058.6568603515625, 18233.23046875, 1099.9212646484375, 891.7163696289062, 4566.39990234375, 767.3123779296875, 1045.271728515625, 1623.881103515625, 856.7132568359375, 3777.787109375, 1373.5948486328125, 1266.3497314453125, 1810.636962890625, 2190.313232421875, 787.8353881835938, 2011.8814697265625, 1665.766357421875, 1007.9365234375, 2019.712646484375], total_intensity=75014.31701660156, ms_level=2, scan_number=10, precursor_mass=473.740315, file_name='/Users/zacharymcgrath/Desktop/nod2 data/single/singleRealSpectrum.mzml', id='NOD2_E3.11427.11516.2.pkl')
DLQTLEVEs = 'DLQTLEVE'

ppm_tol = 20

# See how well each of these score on each scoring algorithm
* `score_subsequence(pepspec: list, subseq: str, ppm_tolerance=20)`
* `backbone_score(observed: Spectrum, reference: str, ppm_tolerance: int)` 
* `intensity_backbone_score(observed: Spectrum, reference: str, ppm_tolerance: int)`
* `ion_backbone_score(observed: Spectrum, reference: str, ion: str, ppm_tolerance: int)`
* `intensity_ion_backbone_score(observed: Spectrum, reference: str, ion: str, ppm_tolerance: int)`
* `xcorr(observed: np.ndarray, reference: np.ndarray, value=50)`

## score_subsequence

In [3]:
print(f'DEAPNFEANTTIGRIRFH: {scoring.score_subsequence(DEAPNFEANTTIGRIRFH.spectrum, DEAPNFEANTTIGRIRFHs, ppm_tol)}')
print(f'DLQTLEVE: {scoring.score_subsequence(DLQTLEVE.spectrum, DLQTLEVEs, ppm_tol)}')
print(f'DLQTLALWSRM: {scoring.score_subsequence(DLQTLALWSRM.spectrum, DLQTLALWSRMs, ppm_tol)}')
print(f'DLTEYLSRFGEVV: {scoring.score_subsequence(DLTEYLSRFGEVV.spectrum, DLTEYLSRFGEVVs, ppm_tol)}')
print(f'DLQTLEVE: {scoring.score_subsequence(DLQTLEVE.spectrum, DLQTLEVEs, ppm_tol)}')

DEAPNFEANTTIGRIRFH: (2, 5)
DLQTLEVE: (5, 2)
DLQTLALWSRM: (5, 8)
DLTEYLSRFGEVV: (5, 5)
DLQTLEVE: (5, 2)


In [4]:
sum(scoring.score_subsequence(DLQTLALWSRM.spectrum, 'DLQTLASRM', ppm_tol))

6

In [8]:
sorted(gen_spectrum('EVE', ion='y', charge=1)['spectrum'])[-1]

376.171441135

In [9]:
sorted(gen_spectrum('EVE', ion='y', charge=2)['spectrum'])[-1]

188.589358785

In [10]:
s = 'MDPDWGQRDVGWAALLVLFAASLITVLGWMLQYARGLWLSRADGGRDSRPASAAEPGGSLRELGVWRSLLRLRATRTSTPEEAGVRGLLASLFAFKSFRENWQRAWVRALNEQACRDGSSIQIAFEEIPQLPPRASISHVTCVDQSERTMVLHCQLSAEEVRFPISVTQQSPAAVSMETYHVTLTLPPTQLEVSLEEIPDEGLLVSWAFTDRPELSLKVLPKLQTRERDEEQPELSTVEELIKDAIVSTQPAMMVNLRACSAPGGLVPSEKPPTMSQAQPSIPRPTRLFLRQLRASHLGSELGGTEELCCAAELDNPMQQKWTKPMRAGPEVEWTEDLALDLGPQSRELTLKVLRSSSCGDAELLGQATLPVGSPSRPMSRRQVCPLTPGPGKSLSPAATVTAELHYEQGSPRNLGTPTSSTPRPSITPTKKIELDRTIMPDGTVVTTVTTVQSRPRVDGKLDSPSRSPSKVEVTEKMTTVLSESSGPSNASHSSSRESHLSNGLDPVAETAIRQLTEPSGRAAKKTPTKRSTLIISGVSKVPIAQDELALSLGYAASLEASMQDDAGTSGGPSSPPSDPSATSPGPVDALSSPTSVQEADETTRSDISERPSVDDVESETGSTGALETRSLKDHKVSFLRSGTKLIFRRRPRQKEAGLSQSHDDLSNTTATPSVRKKAGSFSRRLIKRFSFKSKPKANGNPSPQL'


In [ ]:
 # go through the kmers for the b sequences
for j in range(len(s) - 3):

    # make a kmer sequence. Do the max (to generate the kmer spec once) then 
    # just iterate through it
    kmer_len = 20 if j + 20 <= len(s) else len(s) - j
    kmer = s[i:i+kmer_len]

    # generate the singly and doubly b spectra
    kmer_spec_b_s = gen_spectrum(kmer, ion='b', charge=1)['spectrum']
    kmer_spec_b_d = gen_spectrum(kmer, ion='b', charge=2)['spectrum']

    # iterate through the spectra and add the entry to the table
    for k in range(min_peptide_len, kmer_len):

        if 'b' in kmer_tracker[kmer[:k]]:
            continue

        kmer_tracker[kmer[:k]] += 'b'

        bs[math.floor(kmer_spec_b_s[k-1])].append(MassSequence(kmer_spec_b_s[k-1], kmer[:k]))
        bd[math.floor(kmer_spec_b_d[k-1])].append(MassSequence(kmer_spec_b_d[k-1], kmer[:k]))